In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime

class TaiwanStockData:
    def __init__(self, date_str):
        """
        初始化股票資料對象

        Args:
            date_str: 日期字串，格式為民國年月日 (例如: '1140731')
        """
        self.date_str = date_str
        self.df = None

    def get_stock_data(self):
        """
        取得台灣股票基本面資訊
        """
        url = f"https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date={self.date_str}&selectType=ALL&response=json&_=1753718400000"
        headers = {'User-Agent': 'Mozilla/5.0'}

        try:
            print(f"正在取得 {self.date_str} 的台灣股票資訊...")
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # 檢查HTTP錯誤

            # 解析JSON資料
            data = response.json()

            if not data:
                print("未取得任何資料")
                return None

            # 轉換為DataFrame
            self.df = pd.DataFrame(data["data"], columns=data["fields"])
            date = data["date"]
            print(date)
            # 資料清理和轉換
            self.df["收盤價"] = pd.to_numeric(self.df["收盤價"].str.replace(",", ""), errors="coerce")
            self.df['殖利率(%)'] = pd.to_numeric(self.df['殖利率(%)'], errors='coerce')
            self.df['本益比'] = pd.to_numeric(self.df['本益比'], errors='coerce')
            self.df['股價淨值比'] = pd.to_numeric(self.df['股價淨值比'], errors='coerce')

            # 添加資料取得時間
            self.df['DataDate'] = date

            print(f"成功取得 {len(self.df)} 筆股票資料")
            print(self.df)
            return self.df

        except requests.exceptions.RequestException as e:
            print(f"網路請求錯誤: {e}")
        except json.JSONDecodeError as e:
            print(f"JSON解析錯誤: {e}")
        except Exception as e:
            print(f"發生未預期錯誤: {e}")
        return None

    def analyze_stock_data(self):
        """
        分析股票資料並顯示統計資訊
        """
        if self.df is None or self.df.empty:
            print("無資料可分析")
            return

        print("\n=== 股票資料統計分析 ===")
        print(f"總股票數量: {len(self.df)}")
        print(f"有P/E比資料的股票: {self.df['本益比'].notna().sum()}")
        print(f"有股息殖利率資料的股票: {self.df['殖利率(%)'].notna().sum()}")
        print(f"有P/B比資料的股票: {self.df['股價淨值比'].notna().sum()}")

        # 顯示基本統計殖利率(%)
        print("\n=== 數值欄位統計 ===")
        numeric_stats = self.df[['本益比', '殖利率(%)', '股價淨值比']].describe()
        print(numeric_stats)
    def analyze_stock_data_2(self):
        if self.df is None or self.df.empty:
            print("無資料可分析")
            return

        # 顯示高股息殖利率股票 (前10名)
        high_dividend = self.df[self.df['殖利率(%)'].notna()].nlargest(10, '殖利率(%)')
        print("\n=== 高股息殖利率股票 (前10名) ===")
        print(high_dividend[['證券代號', '證券名稱', '殖利率(%)', '本益比', '股價淨值比']].to_string(index=False))
        return high_dividend[['證券代號', '證券名稱', '殖利率(%)', '本益比', '股價淨值比']].to_string(index=False)
    def analyze_stock_data_3(self):
        if self.df is None or self.df.empty:
            print("無資料可分析")
            return
        # 顯示低P/E比股票 (前10名，排除負值和異常值)
        low_pe = self.df[(self.df['本益比'] > 0) & (self.df['本益比'] < 100)].nsmallest(10, '本益比')
        print("\n=== 低P/E比股票 (前10名) ===")
        print(low_pe[['證券代號', '證券名稱', '殖利率(%)', '本益比', '股價淨值比']].to_string(index=False))
        return low_pe[['證券代號', '證券名稱', '殖利率(%)', '本益比', '股價淨值比']].to_string(index=False)
    def save_to_excel(self, filename):
        """
        將資料儲存為Excel檔案

        Args:
            filename: 檔案名稱
        """
        if self.df is None or self.df.empty:
            print("無資料可儲存")
            return

        try:
            self.df.to_excel(filename, index=False, engine='openpyxl')
            print(f"\n資料已儲存至: {filename}")
        except Exception as e:
            print(f"儲存檔案時發生錯誤: {e}")

    def search_stock(self, code_or_name):
        """
        搜尋特定股票

        Args:
            code_or_name: 股票代碼或名稱
        """
        if self.df is None:
            print("尚未取得資料")
            return None

        result = self.df[
            (self.df['證券代號'].str.contains(str(code_or_name), na=False)) |
            (self.df['證券名稱'].str.contains(str(code_or_name), na=False))
        ]
        return result

    def get_specific_stock_info(self, stock_code):
        """
        顯示特定股票資訊 (例如台積電)

        Args:
            stock_code: 股票代碼
        """
        text=""
        if self.df is None:
            print("尚未取得資料")
            return None

        stock = self.df[self.df['證券代號'] == stock_code]
        if not stock.empty:
            text = f"\n=== 股票代碼 {stock_code} 資訊 ===\n" + \
               f"公司名稱: {stock.iloc[0]['證券名稱']}\n" + \
               f"收盤價: {stock.iloc[0]['收盤價']}\n" + \
               f"股息殖利率: {stock.iloc[0]['殖利率(%)']}%\n"+ \
               f"本益比: {stock.iloc[0]['本益比']}\n" + \
               f"股價淨值比: {stock.iloc[0]['股價淨值比']}\n" + \
               f"資料日期: {stock.iloc[0]['DataDate']}"

            print(f"\n=== 股票代碼 {stock_code} 資訊 ===")
            print(f"股票代碼: {stock.iloc[0]['證券代號']}")
            print(f"公司名稱: {stock.iloc[0]['證券名稱']}")
            print(f"P/E比: {stock.iloc[0]['本益比']}")
            print(f"股息殖利率: {stock.iloc[0]['殖利率(%)']}%")
            print(f"P/B比: {stock.iloc[0]['股價淨值比']}")
        else:
            text =f"無法找到股票代碼 {stock_code} 的資料"
            print(f"無法找到股票代碼 {stock_code} 的資料")

        return text
    @staticmethod
    def compare_dates(date1, date2):
        """
        比較兩個不同日期的股票資料

        Args:
            date1, date2: 日期字串
        """
        print(f"\n=== 比較 {date1} 與 {date2} 的資料 ===")

        stock_data1 = TaiwanStockData(date1)
        stock_data2 = TaiwanStockData(date2)

        df1 = stock_data1.get_stock_data()
        df2 = stock_data2.get_stock_data()

        if df1 is not None and df2 is not None:
            # 合併資料進行比較
            comparison = df1.merge(df2, on=['證券代號', '證券名稱'], suffixes=('_1', '_2'))

            # 計算變化
            comparison['PE_Change'] = comparison['本益比_2'] - comparison['本益比_1']
            comparison['Dividend_Change'] = comparison['殖利率(%)_2'] - comparison['殖利率(%)_1']

            print(f"可比較的股票數量: {len(comparison)}")

            # 顯示P/E比變化最大的股票
            pe_changes = comparison.dropna(subset=['PE_Change']).nlargest(5, 'PE_Change')
            print("\nP/E比增加最多的股票:")
            print(pe_changes[['證券代號', '證券名稱', '本益比_1', '本益比_2', 'PE_Change']].to_string(index=False))




In [ ]:
mes1 = {
  "type": "text",
  "text": "您好！\n我是股票資訊查詢小幫手,請輸入您欲查詢的代碼[1]或[2]或[3]\n1: 個股日本益比、殖利率及股價淨值比\n2. 高股息殖利率股票 (前10名)\n3. 低本益比股票 (前10名)\n"
}
mes1_1 = {
  "type": "text",
  "text": "查詢單一股票,請回覆4碼股票代碼\n欲查詢特定日期的股票資訊請輸入[5]"
}
mes5 = {
  "type": "text",
  "text": "請依格式輸入:\n股票代碼:XXXX\n日期:20250731"
}
mes_err = {
  "type": "text",
  "text": "無法辨識您的代碼,請重新輸入"
}

In [ ]:
"""
HTTP呼叫模式
參考Message API資料，以request呼叫
https://developers.line.biz/en/reference/messaging-api/#send-reply-message
"""
import requests
import os
from flask import Flask, request,Response
from datetime import datetime

# 初始化 Flask 應用程式
app = Flask(__name__)

# 設定 LINE BOT Token
BOT_TOKEN = "CBfAXWIbkwr8WspmrMja51jwPo6drAn6Zsi5FK6xbHWcsRfLC1Z1njQ/EmrhgdoPEm5HqHbRWbDVbadJ4DJlDzGhq5PfELv+edlt0zn6dU/G6BN7/xB1EDPqMJEhJ5yzR009XpwCeOwOd1EIlmAKwAdB04t89/1O/w1cDnyilFU="

# 傳送訊息函數
def send_text_message(reply_token,meg):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + BOT_TOKEN
    }

    payload = {
        "replyToken": reply_token, "messages": [meg]
     #   "messages": [mes1,mes2,mes3]
    }

    # 發送 POST 請求至 LINE Messaging API
    response = requests.post(
        "https://api.line.me/v2/bot/message/reply", #host 不同資料內容可能不同
        headers=headers,
        json=payload
    )
    return response

# LINE Webhook 入口
@app.route("/", methods=['POST'])
def linebot():
    # 取得使用者傳來的資料
    data = request.get_json()
    print(data)

    # 提取 replyToken
    reply_token = data['events'][0]['replyToken']
    try:
    # 提取使用者問題
        message = data['events'][0]['message']['text']

        target_date = ''
        if message == '1':
            meg = mes1_1
        elif message == '2':
            print(message)
            # 初始化 TaiwanStockData 類別，傳入日期參數
            stock_data = TaiwanStockData(target_date)
            # 取得股票資料
            stock_df = stock_data.get_stock_data()
            mes_return = {
                  "type": "text",
                  "text":  stock_data.analyze_stock_data_2()
            }
            meg =  mes_return
        elif message == '3':
            # 初始化 TaiwanStockData 類別，傳入日期參數
            stock_data = TaiwanStockData(target_date)
        # 取得股票資料
            stock_df = stock_data.get_stock_data()

            #stock_data.analyze_stock_data_3()

            mes_return = {
              "type": "text",
              "text": stock_data.analyze_stock_data_3()
            }
            meg =  mes_return

        elif message == '5':
            meg = mes5
        else:
            if len(message) == 4 and message.isdigit():
            # 設定要取得的股票資料日期
                #today = datetime.today()
                #target_date = today.strftime('%Y%m%d')
                print(target_date)
            # 初始化 TaiwanStockData 類別，傳入日期參數
                stock_data = TaiwanStockData(target_date)
            # 取得股票資料
                stock_df = stock_data.get_stock_data()

                #stock_data.get_specific_stock_info(message)

                mes_return = {
                  "type": "text",
                  "text": stock_data.get_specific_stock_info(message)
                }
                meg =  mes_return
            else:
                #message = '股票代碼:2330\n日期:20250731'
                stock_code = ""
                date = ""
                if ("股票代碼" in message and "日期" in message):
                    lines = message.split('\n')
                    for x in lines:
                        if "股票代碼" in x:
                            stock_code = x.split(':')[1]
                        if "日期" in x:
                            date = x.split(':')[1]

                    if is_valid_date(date):
                        # 設定要取得的股票資料日期
                        #today = datetime.today()
                        #target_date = today.strftime('%Y%m%d')
                        target_date=date
                # 初始化 TaiwanStockData 類別，傳入日期參數
                        stock_data = TaiwanStockData(target_date)
                # 取得股票資料
                        stock_df = stock_data.get_stock_data()
                        mes_return = {
                          "type": "text",
                          "text": stock_data.get_specific_stock_info(stock_code)
                        }

                    else:
                        mes_return = {
                          "type": "text",
                          "text": "日期或格式錯誤"
                        }
                    meg =  mes_return
                else:
                    meg =mes1
    except Exception as e:
        print(f"發生未預期錯誤: {e}")
        mes_return = {
            "type": "text",
            "text": f"發生未預期錯誤: {e}"
            }
        meg = mes_return
   # 回傳文字訊息
    response = send_text_message(reply_token,meg)

    if response.status_code == 200:
        return "OK", 200
    else:
        print("發送訊息失敗:", response.status_code, response.text)
        return "Error", 400

def is_valid_date(date_str, date_format="%Y%m%d"):
    try:
        datetime.strptime(date_str, date_format)
        return True
    except ValueError:
        return False

# 啟動 Flask 伺服器
if __name__ == "__main__":
    app.run(port=3000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3000
Press CTRL+C to quit


{'destination': 'Ub877702c2cab878b4bf24118bbc595f1', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '572704330688495652', 'quoteToken': '6kl6V3HAcRq4pHIslHl1CCZrUP9oshrxK1Ypu_B7zT7K041SYhqXty9kq7e5dWIhOIXjEqlmdfmcjqLyuch29ura7uMf0PVZOMG0QrScTFPFzkzqtkwFZvqXsqEBHaJPrdSnHmY4rEstkxmmSkgrvw', 'text': 'hihi'}, 'webhookEventId': '01K1PXMSDW8J2FYST45MGA30PA', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1754189686002, 'source': {'type': 'user', 'userId': 'U0d3e230f03f69a8ca81ddc5020db6a4e'}, 'replyToken': 'e16bc08596734bc6a9ba36daabd6f6bd', 'mode': 'active'}]}


127.0.0.1 - - [03/Aug/2025 10:54:46] "POST / HTTP/1.1" 200 -


{'destination': 'Ub877702c2cab878b4bf24118bbc595f1', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '572704350485348714', 'quoteToken': 'sMCdNHA6tveZdd5A07CsUBtOmugYmQFfmrp-v6jJ6aDDAwbPDrQCl7BCHQRsAruI-P1mk6GBFSjnauw0oJ-NRtkHPP-fFA7D8jJGa67dsTgbpHa6odyo_N3T4EUHeI2RP_pQmRFtNyi37kySUMJBog', 'text': '2'}, 'webhookEventId': '01K1PXN563HZKNG1TJM25SJ2QX', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1754189697737, 'source': {'type': 'user', 'userId': 'U0d3e230f03f69a8ca81ddc5020db6a4e'}, 'replyToken': '44146f81fe2348688b82f5351917cf64', 'mode': 'active'}]}
2
正在取得  的台灣股票資訊...
20250801
成功取得 1046 筆股票資料
      證券代號  證券名稱     收盤價  殖利率(%)  股利年度    本益比  股價淨值比  財報年/季  DataDate
0     1101    台泥   24.15    4.14   113  19.02   0.73  114/1  20250801
1     1102    亞泥   40.15    5.48   113  12.59   0.80  114/1  20250801
2     1103    嘉泥   13.90    3.96   113  20.75   0.43  114/1  20250801
3     1104    環泥   26.85    6.33   113  12.15   0.77  114/1  20250801
4     1108    幸福   

...

127.0.0.1 - - [03/Aug/2025 10:56:18] "POST / HTTP/1.1" 200 -


{'destination': 'Ub877702c2cab878b4bf24118bbc595f1', 'events': [{'type': 'message', 'message': {'type': 'sticker', 'id': '572704598167388751', 'quoteToken': 'EEdltvUID5QDIawJKKHsDBtfToqtwwUOEoAP4WXYdiRnN30rpF3MRLkP3-1hBRvIM5LnkImUJg999GN1XIgTzgcsx6MX6ahBzNRpkqDOMcuHR6x6qkVImHymNeTWPV1stxjjDDuXh9qjAXNZi1ZOSA', 'stickerId': '585913', 'packageId': '1012965', 'stickerResourceType': 'STATIC', 'keywords': ['Hello', 'Good morning', 'hey', 'Bye']}, 'webhookEventId': '01K1PXSNC6G4059EZRVV5XBT31', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1754189845370, 'source': {'type': 'user', 'userId': 'U0d3e230f03f69a8ca81ddc5020db6a4e'}, 'replyToken': 'a7787606a91345fb825a74266b22a173', 'mode': 'active'}]}
發生未預期錯誤: 'text'


127.0.0.1 - - [03/Aug/2025 10:57:26] "POST / HTTP/1.1" 200 -


{'destination': 'Ub877702c2cab878b4bf24118bbc595f1', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '572704621941489802', 'quoteToken': 'IF4blS-l9p7hjPIPUrd_zyZ7WZgHRYzHn-bJqSar3qIFUIkOneMmCKx82J0G-zCzoBgQzmc_FZRGCwyFNVRLcbStgL6Y2NSicdneFyBvsSX667QANmW3dY_E7Q8KUumKEAeu9W0iRRK7YhgBnLU5Gg', 'text': '111'}, 'webhookEventId': '01K1PXT36AR5BSMKXN4CA3NQCS', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1754189859608, 'source': {'type': 'user', 'userId': 'U0d3e230f03f69a8ca81ddc5020db6a4e'}, 'replyToken': 'c2d6f1856b624517b2abd0771a369106', 'mode': 'active'}]}


127.0.0.1 - - [03/Aug/2025 10:57:40] "POST / HTTP/1.1" 200 -
